# Results evaluation
by: Kaike Wesley Reis

## Modules

In [1]:
# Standard modules
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

# Graphical modules
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Models
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Bootstrap
from sklearn.utils import resample
# Evaluation
from scipy import stats
from sklearn.metrics import classification_report, confusion_matrix, brier_score_loss
from sklearn.metrics import f1_score,roc_auc_score,recall_score, precision_score, accuracy_score, balanced_accuracy_score

# Import models
import joblib

# Import data & models

In [2]:
# Models
mdl_lre = joblib.load('results_modelsDevelopment/model_lre_oversampled.sav')
mdl_svm = joblib.load('results_modelsDevelopment/model_svm_oversampled.sav')
mdl_rfc = joblib.load('results_modelsDevelopment/model_rfc_oversampled.sav')
mdl_bst = joblib.load('results_modelsDevelopment/model_bst_oversampled.sav')
mdl_xgb = joblib.load('results_modelsDevelopment/model_xgb_oversampled.sav')
# Dummy models
mdl_dmf = joblib.load('results_modelsDevelopment/model_dummy_mf_oversampled.sav')
mdl_dst = joblib.load('results_modelsDevelopment/model_dummy_st_oversampled.sav')
# Testset
x_test = pd.read_csv('results_modelsDevelopment/x_test.csv')
y_test = pd.read_csv('results_modelsDevelopment/y_test.csv')
# BOCV-5 results
bocv5_results = pd.read_csv('results_modelsDevelopment/BO5CV_best_results.csv')
# Backtest
x_resp = pd.read_csv('results_modelsDevelopment/x_resp.csv')
y_resp = pd.Series(np.zeros((len(x_resp),)))

# Auxilar functions & Parameters

In [3]:
# Bootstrap parameters
REPETITIONS = 999
SAMPLE_SIZE = len(y_test)
RS_GENERATOR = range(0,REPETITIONS)

In [4]:
def calculate_metrics(model, x_test, y_true):
    # Generate a prediction using the model
    y_pred = model.predict(x_test)
    # Calculate metrics
    f1s = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    # return results
    return {'F1-Score':round(100*f1s,3),'AUC ROC':round(100*auc,3),'Accuracy':round(100*acc,3)}

In [5]:
def calculate_confusion_matrix_results(model, x_test, y_true):
    # Generate a prediction using the model
    y_pred = model.predict(x_test)
    
    # Calculate a confusion matrix to retrieve the binary CM values
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Return
    return {'TN':tn,'TP':tp,'FN':fn,'FP':fp}

In [6]:
def calculate_1_minus_brier_score_loss(model, x_test, y_true):
    y_pred = model.predict_proba(x_test)[:,1]
    prob_score = brier_score_loss(y_true, y_pred, pos_label=1)
    return {'Brier Score':round(prob_score, 3), '1 - Brier Score':round(1-prob_score, 3)}

In [7]:
def bootstrap_resampling(x_test, y_test, rs_number, sample_size=SAMPLE_SIZE):
    # Generate X sample
    bootstrap_x = resample(x_test, replace=True, n_samples=sample_size, random_state=rs_number)
    # Get index for X to get Y value
    bootstrap_y = y_test.loc[bootstrap_x.index]
    # Return
    return bootstrap_x, bootstrap_y

In [8]:
def bootstrap_confidence_interval(values):
    percents = np.percentile(values, [2.5, 97.5])
    lower_bound = round(max(0.0, percents[0]), 3)
    upper_bound = round(min(1.0, percents[1]), 3)
    mean_value = round(np.mean(values), 3)
    return (lower_bound, mean_value, upper_bound)

In [9]:
def calculate_metrics_best_model(model, x_test, y_true):
    # Generate a prediction using the model
    y_pred = model.predict(x_test)
       
    # Calculate several metrics with SKlearn
    f1score = f1_score(y_true, y_pred) #
    roc_auc = roc_auc_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred,pos_label=1)
    specificity = recall_score(y_true, y_pred,pos_label=0)
    precision = precision_score(y_true, y_pred)

    # Return
    return f1score, roc_auc, sensitivity, specificity, precision  

In [10]:
def calculate_backtest_accuracy(model, x_resp, y_resp):
    # Generate a prediction using the model
    y_pred = model.predict(x_resp)
    # Calculate accuracy
    acc = accuracy_score(y_resp, y_pred)
    # return
    return acc

In [11]:
def calculate_backtest_real_accuracy(mdl_object, x_resp, y_resp):
    return {'ACC (%)':100*accuracy_score(y_resp.values.ravel(),mdl_object.predict(x_resp)).round(2)}

In [12]:
def get_metrics_using_bootstrap_best_model(model, x_test, y_test, rs_generator):
    # Metrics list
    f1s_list = []
    roc_list = []
    sen_list = []
    spe_list = []
    pre_list = []
    
    # Loop to generate a sample and generate metrics
    for rs in rs_generator:
        x_sample, y_sample = bootstrap_resampling(x_test, y_test, rs_number=rs)
        f1s, roc, sen, spe, pre = calculate_metrics_best_model(model, x_sample, y_sample)
        # Append results
        f1s_list.append(f1s)
        roc_list.append(roc)
        sen_list.append(sen)
        spe_list.append(spe)
        pre_list.append(pre)
    
    # Return
    return {'F1S':f1s_list,'ROC':roc_list,'SEN':sen_list,'SPE':spe_list,'PRE':pre_list}

In [13]:
def get_metrics_using_bootstrap_for_backtest(model, x_resp, y_resp, rs_generator):
    # Metrics list
    acc_list = []
    
    # Loop to generate a sample and generate metrics
    for rs in rs_generator:
        x_sample, y_sample = bootstrap_resampling(x_resp, y_resp, rs_number=rs)
        acc = calculate_backtest_accuracy(model, x_sample, y_sample)
        # Append results
        acc_list.append(acc)
    
    # Return
    return acc_list

In [14]:
def eval_pvalue(pval):
    if pval > 0.05:
        return 'No significant difference between distributions (fail to reject H0)'
    else:
        return 'Different distributions (reject H0)'

# EVALUATION 0 - Model Development results

In [15]:
# Rounding up
bocv5_results['mean_test_score'] = np.round(100*bocv5_results['mean_test_score'],3)
bocv5_results['std_test_score'] = np.round(100*bocv5_results['std_test_score'],3)

# Show
bocv5_results[['mean_test_score','std_test_score','ai_algorithm']]

,mean_test_score,std_test_score,ai_algorithm
0,83.442,3.080,lre
1,94.785,1.352,svm
2,94.561,2.021,rfc
3,96.024,1.987,bst
4,93.683,2.448,xgb


# EVALUATION 1 - Calculate several metrics for test set 

**Metrics**
- F1 Score (related to precision and recall)
- AUC ROC Score
- Accuracy

## Test Overall results

In [16]:
# List to keep our results
test_results = []
# Loop to calculate test results
for mdl_object, mdl_name in zip([mdl_lre,mdl_svm,mdl_rfc,mdl_bst,mdl_xgb,mdl_dst],['LRE','SVM','RFC','BST','XGB','DST']):
    test_result = calculate_metrics(mdl_object, x_test, y_test)
    test_result['Model'] = mdl_name
    test_results.append(test_result)

# Transform into a dataframe
df_test_results = pd.DataFrame(test_results)
df_test_results

,F1-Score,AUC ROC,Accuracy,Model
0,48.649,77.244,78.889,LRE
1,51.852,74.038,85.556,SVM
2,78.261,86.218,94.444,RFC
3,57.143,73.077,90.000,BST
4,58.333,75.962,88.889,XGB
5,26.923,58.013,57.778,DST


# EVALUATION 2 - Get Confusion Matrix results from test set

In [17]:
# List to keep our CM results
test_results = []
# Loop to calculate CM results
for mdl_object, mdl_name in zip([mdl_lre,mdl_svm,mdl_rfc,mdl_bst,mdl_xgb,mdl_dst],['LRE','SVM','RFC','BST','XGB','DST']):
    test_result = calculate_confusion_matrix_results(mdl_object, x_test, y_test)
    test_result['Model'] = mdl_name
    test_results.append(test_result)

# Transform into a dataframe
df_cm_results = pd.DataFrame(test_results)
df_cm_results

,TN,TP,FN,FP,Model
0,62,9,3,16,LRE
1,70,7,5,8,SVM
2,76,9,3,2,RFC
3,75,6,6,3,BST
4,73,7,5,5,XGB
5,31,7,5,47,DST


# EVALUATION 3 - Get a probabilistic metric approach

In [18]:
# List to keep our CM results
test_results = []
# Loop to calculate CM results
for mdl_object, mdl_name in zip([mdl_lre,mdl_svm,mdl_rfc,mdl_bst,mdl_xgb,mdl_dst],['LRE','SVM','RFC','BST','XGB','DST']):
    test_result = calculate_1_minus_brier_score_loss(mdl_object, x_test, y_test)
    test_result['Model'] = mdl_name
    test_results.append(test_result)

# Transform into a dataframe
df_brier_results = pd.DataFrame(test_results)
df_brier_results

,Brier Score,1 - Brier Score,Model
0,0.139,0.861,LRE
1,0.104,0.896,SVM
2,0.073,0.927,RFC
3,0.091,0.909,BST
4,0.081,0.919,XGB
5,0.422,0.578,DST


# EVALUATION 5 - Backtests Accuracy

In [19]:
# List to keep our CM results
test_results = []
# Loop to calculate CM results
for mdl_object, mdl_name in zip([mdl_lre,mdl_svm,mdl_rfc,mdl_bst,mdl_xgb,mdl_dst],['LRE','SVM','RFC','BST','XGB','DST']):
    test_result = calculate_backtest_real_accuracy(mdl_object, x_resp, y_resp)
    test_result['Model'] = mdl_name
    test_results.append(test_result)

# Transform into a dataframe
df_backtest_results = pd.DataFrame(test_results)
df_backtest_results

,ACC (%),Model
0,85.0,LRE
1,85.0,SVM
2,92.0,RFC
3,96.0,BST
4,96.0,XGB
5,46.0,DST


# Overall Commentary over previous results
As you can see **Random Forest** performs better compared to the others (considering the best results into CM/Metrics/Brier Score):
- Metrics: Best F1-Score and AUC
- CM: Best TN/TP/FP/FN
- Best Brier Score
- Lose by one missclassified sample

The only result where RF was not the best, by one sample, was backtest evaluation. So the selected model will be the RF.

# FINAL RESULT - Bootstrap CI 95% for selected AI algorithm with several metrics
- F1-Score
- AUC ROC
- Sensitivity
- Specificity
- Precision

## Get BCI 95% for RF

In [20]:
# Generate a list of metrics using Bootstrap
selected_model_bci_results = get_metrics_using_bootstrap_best_model(mdl_rfc, x_test, y_test, rs_generator=RS_GENERATOR)

In [21]:
# Create dataframe result
df_bci_best_model = pd.DataFrame(index=[0],columns=list(selected_model_bci_results.keys()))

In [22]:
# Calculate BCI95%
for i in df_bci_best_model.columns:
    df_bci_best_model.loc[0, i] = bootstrap_confidence_interval(selected_model_bci_results[i])

# Show results
df_bci_best_model

,F1S,ROC,SEN,SPE,PRE
0,"(0.545, 0.775, 0.941)","(0.719, 0.862, 0.988)","(0.455, 0.75, 1.0)","(0.936, 0.975, 1.0)","(0.545, 0.821, 1.0)"


In [23]:
# Show Evaluation 1 results for F1S and ROC
df_test_results.loc[df_test_results['Model'] == 'RFC']

,F1-Score,AUC ROC,Accuracy,Model
2,78.261,86.218,94.444,RFC


# **Export results**

In [24]:
df_test_results.to_csv('results_resultsEval/eval_1_testSet_metrics.csv',index=False)
df_cm_results.to_csv('results_resultsEval/eval_2_confusionMatrix_metrics.csv',index=False)
df_brier_results.to_csv('results_resultsEval/eval_3_testSet_brierscore.csv',index=False)
df_backtest_results.to_csv('results_resultsEval/eval_4_backtest.csv',index=False)
df_bci_best_model.to_csv('results_resultsEval/best_model_bci_result.csv',index=False)